In [22]:
import pandas as pd
import os
import requests
import pandas as pd
from typing import List, Dict

In [35]:
data_processed_path = os.path.join('..','data','processed')
data_cleaned_path = os.path.join('..','data','cleaned')
stations_lithuania = pd.read_csv('../notes/lithuanian_road_distances.csv')


In [48]:
class LithuanianWeatherAPI:
    """API meteo.lt"""

    def __init__(self):
        self.base_url = "https://api.meteo.lt/v1"
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'StationListDownloader/1.0',
            'Accept': 'application/json'
        })

    def get_stations(self) -> List[Dict]:
        """List of all meteostation"""
        print("Downloading the list of stations...")
        try:
            response = self.session.get(f"{self.base_url}/stations")
            response.raise_for_status()  # Raise an exception for bad status codes
            stations = response.json()
            print(f"Found {len(stations)} stations.")
            return stations
        except requests.exceptions.RequestException as e:
            print(f"Error downloading stations: {e}")
            return []

def download_and_save_station_list():
    api = LithuanianWeatherAPI()
    stations_raw = api.get_stations()

    if not stations_raw:
        print("Could not retrieve station list. Exiting.")
        return

    print("\nProcessing station data...")
    stations_processed = []
    for station in stations_raw:
        # Використовуємо .get() для безпечного доступу до даних
        coords = station.get('coordinates', {})
        stations_processed.append({
            'code': station.get('code'),
            'name': station.get('name'),
            'latitude': coords.get('latitude'),
            'longitude': coords.get('longitude')
        })

    df = pd.DataFrame(stations_processed)
    output_filename = '../data/raw/lithuanian_weather_stations.csv'
    try:
        df.to_csv(output_filename, encoding='utf-8-sig')
        print(f"Station list successfully saved to: {output_filename}")
    except Exception as e:
        print(f"Failed to save file: {e}")

    print("\n List of Weather Stations and Coordinates:")
    print(df)


if __name__ == "__main__":
    download_and_save_station_list()

Found 52 stations.

Processing station data...
Station list successfully saved to: ../data/raw/lithuanian_weather_stations.csv

 List of Weather Stations and Coordinates:
                code              name   latitude  longitude
0        akmenes-ams       Akmenės AMS  56.249920  22.730810
1        alytaus-ams       Alytaus AMS  54.412435  24.063274
2       anyksciu-ams      Anykščių AMS  55.517350  25.117800
3       birstono-ams      Birštono AMS  54.603217  24.035802
4          birzu-ams         Biržų AMS  56.193191  24.774184
5       dotnuvos-ams      Dotnuvos AMS  55.395993  23.866224
6    druskininku-ams   Druskininkų AMS  54.018202  23.980791
7         duksto-ams        Dūkšto AMS  55.517856  26.316140
8      elektrenu-ams     Elektrėnų AMS  54.786161  24.689854
9        jonavos-ams       Jonavos AMS  55.069130  24.277147
10      joniskio-ams      Joniškio AMS  56.246354  23.630755
11      jurbarko-ams      Jurbarko AMS  55.094064  22.773518
12   kaisiadoriu-ams   Kaišiadorių A

In [55]:
stations_lithuania1 = pd.read_csv('../data/raw/lithuanian_weather_stations.csv')

In [63]:
df_copy2 = stations_lithuania1.copy()
df_copy2.drop('Unnamed: 0', axis=1, inplace=True)
df_copy2['name'] = df_copy2['name'].str.replace(' AMS', '', regex=False)
df_copy2.head()


,code,name,latitude,longitude
0,akmenes-ams,Akmenės,56.249920,22.730810
1,alytaus-ams,Alytaus,54.412435,24.063274
2,anyksciu-ams,Anykščių,55.517350,25.117800
3,birstono-ams,Birštono,54.603217,24.035802
4,birzu-ams,Biržų,56.193191,24.774184


In [36]:
df_copy1 = df_copy2.copy()
print(df_copy1.columns)

Index(['group_id', 'road_path', 'total_distance_km', 'segment_distances'], dtype='object')


In [37]:
df_copy1['group_number'] = df_copy1['group_id'].str.extract('(\d+)').astype(int)
df_copy1.head()

,group_id,road_path,total_distance_km,segment_distances,group_number
0,Group 1 (Vertical),Nidos -> Ventės -> Klaipėdos -> Kretingos,77.66,Nidos -> Ventės: 12.35 km; Ventės -> Klaipėdos...,1
1,Group 2 (Vertical),Šilutės -> Vėžaičių -> Skuodo,103.07,Šilutės -> Vėžaičių: 41.04 km; Vėžaičių -> Sku...,2
2,Group 3 (Vertical),Plungės -> Rietavo -> Pagėgių,86.06,Plungės -> Rietavo: 21.80 km; Rietavo -> Pagėg...,3
3,Group 4 (Vertical),Laukuvos -> Tauragės -> Telšių -> Mažeikių,161.61,Laukuvos -> Tauragės: 39.27 km; Tauragės -> Te...,4
4,Group 5 (Vertical),Akmenės -> Kelmės -> Jurbarko -> Kybartų,182.03,Akmenės -> Kelmės: 70.29 km; Kelmės -> Jurbark...,5


In [64]:
file_name = 'lithuanian_weather_stations_CLEANED.csv'
full_save_path = os.path.join(data_cleaned_path, file_name)
df_copy2.to_csv(full_save_path, index=False, encoding='utf-8')

file_name = 'lithuania_coordinates_CLEANED.csv'
full_save_path = os.path.join(data_cleaned_path, file_name)
df_copy1.to_csv(full_save_path, index=False, encoding='utf-8')